In [1]:
import pymongo
import datetime
import pandas as pd
import numpy as np
import cuemath_db as cuedb
import psycopg2
from update_from_cuedb import *


# Get Doubt to Call Time

In [6]:
def get_datetime(date):
    # print(type(date))
    if(type(date)==type('str')):
        return datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    else:
        return date - datetime.timedelta(hours=5, minutes=30)

def get_doubt_to_call_time(call_data, dbts_raised, class_start_dt_time, class_end_dt_time, cls_id):
    # Get list of time lapsed between doubt raised and call made.
    dbt_tmstmp_list = []
    dbt_tmstmp_list.append(class_start_dt_time)
    if(not dbts_raised or len(dbts_raised)==0):
        return([], 0)
    dbt_tmstmp_list += [get_datetime(dbt.get('raised_at').get('created_on')) for dbt in dbts_raised.values()]
    call_data_list = sorted(call_data, key=lambda x: x.get('start_time'), reverse=False)
    doubt_to_call_time_list = []
    num_unresolved_dbts = 0
    toggle_dbt_resolved = True
    for idx in range(len(dbt_tmstmp_list)-1):
        dbt_tm = dbt_tmstmp_list[idx]
        next_dbt_tm = dbt_tmstmp_list[idx+1]
        if(not toggle_dbt_resolved):
            num_unresolved_dbts += 1
        toggle_dbt_resolved = False
        for call in call_data_list:
            # print(type(call.get('start_time')), type(dbt_tm), type(next_dbt_tm))
            # print(dbt_tms, call.get('start_time'), next_dbt_tm)
            if(call.get('start_time') > dbt_tm and call.get('start_time') <= next_dbt_tm):
                doubt_to_call_time_list.append((call.get('start_time') - dbt_tm).total_seconds())
                toggle_dbt_resolved = True
                continue
    # print(doubt_to_call_time_list, num_unresolved_dbts)
    return(doubt_to_call_time_list, num_unresolved_dbts)

def get_classroom_metrics(inteldb, student_id_list):
    # inteldb = IntelDB('prod')
    # prg_list= ['G7', 'G8', 'CBSE.G9', 'CBSE.G10', 'ICSE.G9', 'ICSE.G10']
    # students = inteldb.student_collection.collection.find({'current_program':{'$in':prg_list}})
    # student_id_list = [std.get('_id') for std in students]
    classrooms = inteldb.classroom_collection.collection.find({
                                                            'student_id':{'$in':student_id_list},
                                                            'av_data':{'$ne':{}},
                                                            }).sort('date')
    student_dict = {}
    for clsrm in classrooms:
        if(not student_dict.get(clsrm.get('student_id'))):
            student_dict[clsrm.get('student_id')] = {}
            student_dict[clsrm.get('student_id')]['total_call_dur'] = []
            student_dict[clsrm.get('student_id')]['num_dropped_calls'] = []
            student_dict[clsrm.get('student_id')]['doubt_to_call_time'] = []
            student_dict[clsrm.get('student_id')]['num_calls'] = []
            student_dict[clsrm.get('student_id')]['num_doubts'] = []
            student_dict[clsrm.get('student_id')]['num_unresolved_dbts'] = []
            student_dict[clsrm.get('student_id')]['num_sheets_attempted'] = []
            student_dict[clsrm.get('student_id')]['num_items_attempted'] = []
            student_dict[clsrm.get('student_id')]['total_time_on_ws'] = []
            student_dict[clsrm.get('student_id')]['rating'] = []
            student_dict[clsrm.get('student_id')]['classes_attended'] = 0
            student_dict[clsrm.get('student_id')]['classes_total'] = 0
        av_data = clsrm.get('av_data')
        total_call_dur = 0.0
        num_calls = 0
        total_stream_lag = 0.0
        nm_lg_available = 0
        total_time_on_ws = 0.0
        num_sheets_attempted = 0
        num_items_attempted = 0
        class_end_tm = datetime.datetime.strptime(clsrm['scheduled_end_time'], '%H:%M:%S').time()
        class_start_tm = datetime.datetime.strptime(clsrm['scheduled_start_time'], '%H:%M:%S').time()
        class_end_dt_time = datetime.datetime.combine(datetime.datetime.strptime(clsrm['date'], '%Y-%m-%d'), class_end_tm)
        class_start_dt_time = datetime.datetime.combine(datetime.datetime.strptime(clsrm['date'], '%Y-%m-%d'), class_start_tm)
        doubt_to_call_time_list, num_unresolved_dbts = get_doubt_to_call_time(av_data.get('call_data'), av_data.get('doubts_raised'), class_start_dt_time, class_end_dt_time, clsrm.get('_id'))
        if(len(doubt_to_call_time_list)>0):
            mean_doubt_to_call_time = sum(doubt_to_call_time_list)/len(doubt_to_call_time_list)
        else:
            mean_doubt_to_call_time = -1 #Need to choose a better constant.s
        for call in av_data.get('call_data', []):
            total_call_dur += float(call.get('duration_in_seconds'))/60.0
            num_calls += 1
            if(not pd.isnull(call.get('call_stream_lag')) and call.get('call_stream_lag')>0):
                total_stream_lag += call.get('call_stream_lag')
                nm_lg_available += 1

        num_doubts = sum([dbt.get('num_button_press') for dbt in av_data.get('doubts_raised', {}).values()])
        if(nm_lg_available == 0):
            avg_call_lag = 'NA'
        else:
            avg_call_lag = total_stream_lag/nm_lg_available
        for session in clsrm.get('sessions_attempted').values():
            num_items_attempted += session.get('num_attempted_today')
            num_sheets_attempted += 1
            total_time_on_ws += session.get('time_spent')/60.0
        rt = clsrm.get('ratings', {})
        rating = rt.get('rating', 'NA')
        slot = clsrm.get('scheduled_start_time', '') + '-' + clsrm.get('scheduled_end_time', '')
        student_dict[clsrm.get('student_id')]['total_call_dur'].append(total_call_dur)
        student_dict[clsrm.get('student_id')]['num_dropped_calls'].append(av_data.get('dropped_calls', 0))
        student_dict[clsrm.get('student_id')]['doubt_to_call_time'].append(mean_doubt_to_call_time)
        student_dict[clsrm.get('student_id')]['num_unresolved_dbts'].append(num_unresolved_dbts)
        student_dict[clsrm.get('student_id')]['num_calls'].append(num_calls)
        student_dict[clsrm.get('student_id')]['num_doubts'].append(num_doubts)
        student_dict[clsrm.get('student_id')]['num_sheets_attempted'].append(num_sheets_attempted)
        student_dict[clsrm.get('student_id')]['num_items_attempted'].append(num_items_attempted)
        student_dict[clsrm.get('student_id')]['total_time_on_ws'].append(total_time_on_ws)
        student_dict[clsrm.get('student_id')]['rating'].append(rating)
        if(clsrm.get('status')=='P' and num_sheets_attempted>0):
            student_dict[clsrm.get('student_id')]['classes_attended'] += 1
        if(clsrm.get('status')!='TA'):
            student_dict[clsrm.get('student_id')]['classes_total'] += 1
    return student_dict


In [7]:
inteldb = IntelDB('dev')
student_dict = get_classroom_metrics(inteldb, ["13652e02-919e-11e9-b4b6-02420a00f13c", 
                                "ecd43d2e-8f61-11e9-8a15-02420a00f13e",
                                "d13000a6-7324-11e9-9c4d-02420a00faba",
                                "7178e646-97e6-11e9-9465-02420a00f0d3"
                               ])

In [8]:
student_dict

{'d13000a6-7324-11e9-9c4d-02420a00faba': {'total_call_dur': [2.5446924666666666,
   32.2854795,
   23.15230615,
   30.43572456666666,
   15.66768365,
   6.817369933333333,
   27.30637981666667,
   0.0,
   26.87244893333333,
   22.742255233333335,
   16.268156883333337,
   13.486367583333331,
   20.2500557,
   11.930328333333334],
  'num_dropped_calls': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  'doubt_to_call_time': [31639.5795,
   -1,
   73.43549999999999,
   119.654,
   114.621,
   -1,
   -1,
   -1,
   -1,
   -1,
   265.2,
   -1,
   -1,
   -1],
  'num_calls': [3, 6, 6, 8, 5, 5, 7, 0, 5, 4, 7, 5, 4, 3],
  'num_doubts': [1, 2, 3, 4, 2, 0, 2, 0, 1, 6, 2, 1, 2, 1],
  'num_unresolved_dbts': [0, 1, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0],
  'num_sheets_attempted': [1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1],
  'num_items_attempted': [20,
   32,
   15,
   23,
   4,
   53,
   35,
   23,
   17,
   16,
   17,
   10,
   8,
   22],
  'total_time_on_ws': [29.733333333333334,
   48.95,
   58.15,
   

# Create Google Sheet for Student-Activities

In [63]:
import json

from learnosity_sdk.request import DataApi
from config_update_script import *

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from curriculum_google_sheet import initialize_creds

from update_from_cuedb import *
import numpy as np

fl_nm_pickle = './google_sheets/token.pickle'
inteldb = IntelDB('prod')
client = DataApi()

In [64]:
activity_id_list_full = ['CDT-GradeK-content-test',
                         'CDT-Grade1-content-test',
                        'CDT-Grade2-content-test',
                        'CDT-Grade3-content-test',
                        'CDT-Grade4-content-test',
                        'CDT-Grade5-content-test',
                        'CDT-Grade6-content-test',
                        ]
data_request_session_here = {
		"activity_id": activity_id_list_full,
		"sort": "asc",
		"mintime": "2019-06-18",
		"limit": 50
}

In [65]:
def get_option(val):
    if(val=='0'):
        return 'A'
    elif(val=='1'):
        return 'B'
    elif(val=='2'):
        return 'C'
    elif(val=='3'):
        return 'D'
    return None
def get_session_data():
    res = client.results_iter(endpoint_sessions, security, consumer_secret, data_request_session_here, action)
    data_dict = {}
    for idx, session in enumerate(res):
        print("Session #", idx+1, session.get('session_id'))
        if(data_dict.get(session.get('activity_id')) is None):
            data_dict[session.get('activity_id')] = {}
        else:
            for response in session.get('responses'):
                if(data_dict[session.get('activity_id')].get(response.get('item_reference')) is None):
                    data_dict[session.get('activity_id')][response.get('item_reference')] = {'correct':0, 
                                                                                             'incorrect':0,
                                                                                             'unattempted':0,
                                                                                            'total':0,
                                                                                             'correct_option':None,
                                                                                            'A':0,
                                                                                            'B':0,
                                                                                            'C':0,
                                                                                            'D':0}
                student_response = response.get('response')
                if(student_response is not None):
                    option_selected = get_option(student_response.get('value')[0])
                else:
                    option_selected = None
                if(option_selected is None):
                    data_dict[session.get('activity_id')][response.get('item_reference')]['unattempted'] += 1
                else:
                    data_dict[session.get('activity_id')][response.get('item_reference')][option_selected] += 1
                if(response.get('score')==response.get('max_score')):
                    data_dict[session.get('activity_id')][response.get('item_reference')]['correct_option']= option_selected
                    data_dict[session.get('activity_id')][response.get('item_reference')]['correct'] += 1
                else:
                    data_dict[session.get('activity_id')][response.get('item_reference')]['incorrect'] += 1
                data_dict[session.get('activity_id')][response.get('item_reference')]['total'] += 1                    
    output_data=[['Worksheet Reference', 
                  'Item Reference', 
                  'Total attempts',
                  'Correct attempts',
                  'Incorrect attempts',
                  'Unanswered',
                  'Correct option (A/B/C/D)',
                  '#students answering A',
                  '#students answering B',
                  '#students answering C',
                  '#students answering D',
                 ]]
    for activity_id, act_dict in data_dict.items():
        for item_id, item_dict in act_dict.items():
            row = [activity_id, 
                   item_id, 
                   item_dict.get('total'),
                   item_dict.get('correct'),
                    item_dict.get('incorrect'),
                    item_dict.get('unattempted'),
                    item_dict.get('correct_option'),
                    item_dict.get('A'),
                    item_dict.get('B'),
                    item_dict.get('C'),
                    item_dict.get('D'),
                  ]
            output_data.append(row)
    return output_data
        

In [66]:
spreadsheet_id = '144L-l-Pjjs_cQ320XH21-Yg5sgwpgoaVPXhacJMTeRc'
creds = initialize_creds()
service = build('sheets', 'v4', credentials=creds)
data = get_session_data()
final_request = {
            "value_input_option": "USER_ENTERED",
            "data": [{
                    "range": "CDT Learnosity Data!A1:N"+str(len(data)+3),
                    "values": data
                }]
            }
request = service.spreadsheets().values().batchUpdate(spreadsheetId=spreadsheet_id, body=final_request)
response = request.execute()
print('Response', response)

Session # 1 0db8e3d0-a485-11e9-a2ba-02420a00f5f6
Session # 2 718d33e4-a487-11e9-bb00-02420a00f5f6
Session # 3 506290d8-a488-11e9-81c6-02420a00f5f6
Session # 4 19d265b8-a48b-11e9-a62a-02420a00f5f6
Session # 5 9039bcda-a485-11e9-a7d0-02420a00f5f6
Session # 6 7c09a5c8-a48a-11e9-bbd9-02420a00f5f6
Session # 7 abd27a90-a489-11e9-986f-02420a00f5f6
Session # 8 6298fdde-a489-11e9-8f75-02420a00f5f6
Session # 9 326e21c8-a488-11e9-8dd6-02420a00f5f6
Session # 10 a01cba54-a48a-11e9-b85f-02420a00f5f6
Session # 11 c4c32182-a48a-11e9-a8ab-02420a00f5f6
Session # 12 309c3494-a48b-11e9-9b71-02420a00f5f6
Session # 13 31606aa2-a487-11e9-8b69-02420a00f5f6
Session # 14 dc67d6d2-a48a-11e9-8882-02420a00f5f6
Session # 15 74e50786-a485-11e9-ba47-02420a00f5f6
Session # 16 622f26a4-a6ce-11e9-a699-02420a00f611
Session # 17 e15b5988-a79b-11e9-b8be-02420a00f634
Session # 18 748ac0ca-a79d-11e9-bf29-02420a00f634
Session # 19 f6b9c5ba-a79f-11e9-a242-02420a00f634
Session # 20 3f542c48-a7a2-11e9-978c-02420a00f634
Session #

Session # 201 a992e280-a93f-11e9-9875-02420a00f208
Session # 202 ab7b491e-a93f-11e9-b405-02420a00f208
Session # 203 a6295ad8-a93f-11e9-8439-02420a00f208
Session # 204 a8ce4428-a93f-11e9-b336-02420a00f208
Session # 205 a53b495a-a93f-11e9-a6ff-02420a00f208
Session # 206 aaac069e-a93f-11e9-a76f-02420a00f208
Session # 207 a61e10f8-a93f-11e9-8634-02420a00f208
Session # 208 ac7590fe-a93f-11e9-9322-02420a00f208
Session # 209 abf51b74-a93f-11e9-a8f1-02420a00f208
Session # 210 aa6ebc1a-a93f-11e9-a600-02420a00f208
Session # 211 a54f2770-a93f-11e9-991a-02420a00f208
Session # 212 add462c2-a93f-11e9-9ad4-02420a00f208
Session # 213 a7c73c66-a93f-11e9-bbfc-02420a00f208
Session # 214 ab7a52e6-a93f-11e9-a61d-02420a00f208
Session # 215 add97cc2-a93f-11e9-a9c8-02420a00f208
Session # 216 a959d6cc-a93f-11e9-9b3b-02420a00f208
Session # 217 a8846854-a93f-11e9-af62-02420a00f208
Session # 218 ad708dc2-a93f-11e9-8354-02420a00f208
Session # 219 a82c4eac-a93f-11e9-8507-02420a00f208
Session # 220 ab40065c-a93f-11e

Session # 401 a70a0168-a93f-11e9-8a29-02420a00f208
Session # 402 ab3edad2-a93f-11e9-8e01-02420a00f208
Session # 403 aa10b314-a93f-11e9-8e5e-02420a00f208
Session # 404 ac8d3112-a93f-11e9-a86d-02420a00f208
Session # 405 abfe1a9e-a93f-11e9-8a39-02420a00f208
Session # 406 a6aa3f7a-a93f-11e9-ae86-02420a00f208
Session # 407 a8dec728-a93f-11e9-88fb-02420a00f208
Session # 408 aa5bf7fa-a93f-11e9-a2f3-02420a00f208
Session # 409 19e1ad68-a9fe-11e9-a71f-02420a00f684
Session # 410 a7462e2c-a93f-11e9-9a34-02420a00f208
Session # 411 a6695d54-a93f-11e9-baf4-02420a00f208
Session # 412 ac5c0b02-a93f-11e9-8bac-02420a00f208
Session # 413 a702a890-a93f-11e9-b3c1-02420a00f208
Session # 414 aa274a4c-a93f-11e9-9615-02420a00f208
Session # 415 a8189f74-a93f-11e9-be27-02420a00f208
Session # 416 a5dd45d4-a93f-11e9-b7e4-02420a00f208
Session # 417 abc97b98-a93f-11e9-a8f9-02420a00f208
Session # 418 a8962450-a93f-11e9-b436-02420a00f208
Session # 419 a7ee6322-a93f-11e9-a25e-02420a00f208
Session # 420 a7ad4e4a-a93f-11e

Session # 601 ad37dc40-a93f-11e9-8956-02420a00f208
Session # 602 aa286a58-a93f-11e9-a595-02420a00f208
Session # 603 aad98926-a93f-11e9-952e-02420a00f208
Session # 604 a54a7ade-a93f-11e9-9d03-02420a00f208
Session # 605 ac90ca48-a93f-11e9-9a6f-02420a00f208
Session # 606 aa683324-a93f-11e9-ac32-02420a00f208
Session # 607 a564bfca-a93f-11e9-a9d5-02420a00f208
Session # 608 ac058c9e-a93f-11e9-825c-02420a00f208
Session # 609 ad7e24ae-a93f-11e9-9b77-02420a00f208
Session # 610 a717ddac-a93f-11e9-9129-02420a00f208
Session # 611 a7d979ee-a93f-11e9-ab25-02420a00f208
Session # 612 a82e358c-a93f-11e9-8d7e-02420a00f208
Session # 613 a753c200-a93f-11e9-bccb-02420a00f208
Session # 614 a5a9efb6-a93f-11e9-88c6-02420a00f208
Session # 615 96a20182-aa2c-11e9-a1c5-02420a00f684
Session # 616 a783d29e-a93f-11e9-b073-02420a00f208
Session # 617 a4dfd888-a93f-11e9-beca-02420a00f208
Session # 618 a5127708-a93f-11e9-98aa-02420a00f208
Session # 619 a77318c2-a93f-11e9-8a2a-02420a00f208
Session # 620 a49e8a26-a93f-11e

Session # 801 a79ebeb8-a93f-11e9-aafe-02420a00f208
Session # 802 ac91234a-a93f-11e9-b6ea-02420a00f208
Session # 803 a7c7a1f8-a93f-11e9-89b9-02420a00f208
Session # 804 ba5caa6e-adfe-11e9-b4bc-02420a00f756
Session # 805 2b69c228-ade4-11e9-b130-02420a00f754
Session # 806 162bf5d4-ae1f-11e9-8f55-02420a00f754
Session # 807 68006034-adf9-11e9-a36b-02420a00f754
Session # 808 fc8463de-ad14-11e9-b19c-02420a00f747
Session # 809 a5adff46-ae21-11e9-a275-02420a00f755
Session # 810 554f05e4-ac7e-11e9-b494-02420a00f6f6
Session # 811 d2c459e8-ae36-11e9-aca3-02420a00f756
Session # 812 f913d82e-9e58-11e9-9825-0242ac15497e
Session # 813 f913d82e-9e58-11e9-9825-0242ac14597e
Session # 814 f915d82e-9e58-11e9-9825-0242ac14997e
Session # 815 f916d82e-9e58-11e9-9825-0242ac14997e
Session # 816 a8389e74-a93f-11e9-9bab-02420a00f208
Session # 817 a4f90390-a93f-11e9-bcc4-02420a00f208
Session # 818 90cc2d00-ad38-11e9-b8ff-02420a00f74f
Session # 819 9aa9a5f4-ae2c-11e9-a6a2-02420a00f755
Session # 820 a8031a30-a93f-11e

Session # 1001 724dab62-b08e-11e9-ba1b-02420a00f7cd
Session # 1002 a7b3f1b0-a93f-11e9-bf4f-02420a00f208
Session # 1003 69f12efa-b112-11e9-b8c4-02420a00f7cc
Session # 1004 46dbf114-b115-11e9-a147-02420a00f7ce
Session # 1005 71c81c06-b111-11e9-8903-02420a00f7cc
Session # 1006 ce0fe14a-b112-11e9-9ae2-02420a00f7ce
Session # 1007 d1bcc42e-b05f-11e9-92a5-02420a00f7cd
Session # 1008 aa6f1a9a-a93f-11e9-8063-02420a00f208
Session # 1009 42cdf1b6-b126-11e9-9f05-02420a00f7ce
Session # 1010 252ec1cc-b11a-11e9-b2d0-02420a00f7ce
Session # 1011 3b12919e-b129-11e9-97aa-02420a00f7cd
Session # 1012 57816bc2-b0f4-11e9-afc1-02420a00f7ce
Session # 1013 b6e6b91c-b12d-11e9-99b0-02420a00f7cd
Session # 1014 b0ca0254-b12e-11e9-b129-02420a00f7ce
Session # 1015 c1c09836-b12a-11e9-bdfe-02420a00f7ce
Session # 1016 3e35a218-ae1b-11e9-9559-02420a00f756
Session # 1017 a4e31d66-a93f-11e9-aa8d-02420a00f208
Session # 1018 a8869300-a93f-11e9-968e-02420a00f208
Session # 1019 d55e4846-b135-11e9-8840-02420a00f7ce
Session # 10

Session # 1201 6653e2e8-b50d-11e9-9cdc-02420a00f7ce
Session # 1202 694a2ad4-b4d7-11e9-aac9-02420a00f7cc
Session # 1203 8fe8991e-b4ea-11e9-b515-02420a00f7cd
Session # 1204 641bd006-b4d7-11e9-adc2-02420a00f7cd
Session # 1205 8bddd2c0-b369-11e9-a596-02420a00f7cc
Session # 1206 654e7b3a-b4e5-11e9-8d07-02420a00f7ce
Session # 1207 4adda006-b4bd-11e9-9c7b-02420a00f7cc
Session # 1208 0a3cdade-b4f8-11e9-8db2-02420a00f7cc
Session # 1209 71fa063e-b4e3-11e9-9b58-02420a00f7ce
Session # 1210 4dc5b092-b13f-11e9-bcc0-02420a00f7cd
Session # 1211 c584085a-b523-11e9-bae1-02420a00f7cd
Session # 1212 ee0db524-b50a-11e9-8044-02420a00f7ce
Session # 1213 14023a4c-b4fa-11e9-83a9-02420a00f7cc
Session # 1214 8831f7a8-b20c-11e9-bdc6-02420a00f7cd
Session # 1215 7b651a00-b522-11e9-a76d-02420a00f7ce
Session # 1216 f24aaf50-b42c-11e9-b37a-02420a00f7cd
Session # 1217 26020f18-b508-11e9-a1d2-02420a00f7ce
Session # 1218 6f0882b4-b526-11e9-8520-02420a00f7cd
Session # 1219 a88353e4-a93f-11e9-abc3-02420a00f208
Session # 12

Session # 1401 911663f0-b77a-11e9-b5f4-02420a00f7cc
Session # 1402 9684ae42-b77a-11e9-a563-02420a00f7ce
Session # 1403 07a17fdc-b867-11e9-bfdb-02420a00f7ce
Session # 1404 7d7711b0-b8d1-11e9-8508-02420a00f7ce
Session # 1405 399dca34-b82f-11e9-945e-02420a00f7cc
Session # 1406 a74ac19c-a93f-11e9-ae22-02420a00f208
Session # 1407 acf38868-a93f-11e9-ab93-02420a00f208
Session # 1408 a8dccc70-b76d-11e9-b698-02420a00f7cd
Session # 1409 312a781c-b8e6-11e9-8d07-02420a00f7ce
Session # 1410 6e610430-b530-11e9-a44c-02420a00f7ce
Session # 1411 2c0c7db6-b1e2-11e9-a826-02420a00f7ce
Session # 1412 aa49a824-a93f-11e9-9d4f-02420a00f208
Session # 1413 66800fe4-b8fd-11e9-8d01-02420a00f7cc
Session # 1414 08965c28-b8e4-11e9-b2cc-02420a00f7ce
Session # 1415 d8b3b5f8-b8fd-11e9-999f-02420a00f7cd
Session # 1416 21d94bf0-b901-11e9-a401-02420a00f7cc
Session # 1417 a3877c88-b901-11e9-bfe7-02420a00f7ce
Session # 1418 ed55e6e4-b903-11e9-b9f0-02420a00f7cd
Session # 1419 8ea50180-b8ee-11e9-899a-02420a00f7cd
Session # 14